---
title: "Exploratory Data Analytics"
format: html
jupyter: python3
execute:
  echo: true
  warning: false
  error: true
toc: true
---


In [9]:
import pandas as pd

eda = pd.read_csv('data/eda_data.csv')
eda.head()

,COMPANY,LOCATION,POSTED,MIN_EDULEVELS_NAME,MAX_EDULEVELS_NAME,MIN_YEARS_EXPERIENCE,MAX_YEARS_EXPERIENCE,TITLE,SKILLS,SPECIALIZED_SKILLS,...,COMMON_SKILLS,SOFTWARE_SKILLS,SOC_2021_4_NAME,NAICS_2022_6,NAICS2_NAME,REMOTE_TYPE_NAME,SALARY,TITLE_NAME,SKILLS_NAME,SPECIALIZED_SKILLS_NAME
0,894731,"{\n ""lat"": 33.20763,\n ""lon"": -92.6662674\n}",2024-06-02,Bachelor's degree,Master's degree,2.000000,2.000000,ET29C073C03D1F86B4,"[\n ""KS126DB6T061MHD7RTGQ"",\n ""KS126706DPFD3...","[\n ""KS126DB6T061MHD7RTGQ"",\n ""KS128006L3V0H...",...,"[\n ""KS126706DPFD3354M7YK"",\n ""KS1280B68GD79...","[\n ""KS440W865GC4VRBW6LJP"",\n ""KS13USA80NE38...",Data Scientists,441330,Retail Trade,[None],116348.5,Enterprise Analysts,"[\n ""Merchandising"",\n ""Mathematics"",\n ""Pr...","[\n ""Merchandising"",\n ""Predictive Modeling""..."
1,133098,"{\n ""lat"": 44.3106241,\n ""lon"": -69.7794897\n}",2024-06-02,No Education Listed,Master's degree,3.000000,3.000000,ET21DDA63780A7DC09,"[\n ""KS122626T550SLQ7QZ1C"",\n ""KS123YJ6KVWC9...","[\n ""KS122626T550SLQ7QZ1C"",\n ""KS123YJ6KVWC9...",...,[],"[\n ""BGSBF3F508F7F46312E3"",\n ""ESEA839CED378...",Data Scientists,561320,Administrative and Support and Waste Managemen...,Remote,116348.5,Oracle Consultants,"[\n ""Procurement"",\n ""Financial Statements"",...","[\n ""Procurement"",\n ""Financial Statements"",..."
2,39063746,"{\n ""lat"": 32.7766642,\n ""lon"": -96.7969879\n}",2024-06-02,Bachelor's degree,Master's degree,5.000000,3.773903,ET3037E0C947A02404,"[\n ""KS1218W78FGVPVP2KXPX"",\n ""ESF3939CE1F80...","[\n ""ESF3939CE1F80C10C327"",\n ""KS120GV6C72JM...",...,"[\n ""KS1218W78FGVPVP2KXPX"",\n ""BGS1ADAA36DB6...","[\n ""KS126HY6YLTB9R7XJC4Z""\n]",Data Scientists,524291,Finance and Insurance,[None],116348.5,Data Analysts,"[\n ""Management"",\n ""Exception Reporting"",\n...","[\n ""Exception Reporting"",\n ""Data Analysis""..."
3,37615159,"{\n ""lat"": 33.4483771,\n ""lon"": -112.0740373\n}",2024-06-02,No Education Listed,Master's degree,3.000000,3.773903,ET2114E0404BA30075,"[\n ""KS123QX62QYTC4JF38H8"",\n ""KS7G6NP6R6L1H...","[\n ""KS123QX62QYTC4JF38H8"",\n ""KS441PQ64HT13...",...,"[\n ""KS7G6NP6R6L1H1SKFTSY"",\n ""KS1218W78FGVP...","[\n ""KS4409D76NW1S5LNCL18"",\n ""ESC7869CF7378...",Data Scientists,522110,Finance and Insurance,[None],116348.5,Management Analysts,"[\n ""Exit Strategies"",\n ""Reliability"",\n ""...","[\n ""Exit Strategies"",\n ""User Story"",\n ""H..."
4,0,"{\n ""lat"": 37.6392595,\n ""lon"": -120.9970014\n}",2024-06-02,No Education Listed,Master's degree,5.486539,3.773903,ET0000000000000000,[],[],...,[],[],Data Scientists,999999,Unclassified Industry,[None],92500.0,Unclassified,[],[]


In [10]:
# identifying data analyst jobs by keyword searching
keywords = ['Data Analyst', 'Business Analyst', 'Data Engineering', 'Deep Learning',
            'Data Science', 'Data Analysis','Data Analytics',  'Market Research Analyst' 
            'LLM', 'Language Model', 'NLP', 'Natural Language Processing',
            'Computer Vision', 'Business Intelligence Analyst', 'Quantitative Analyst', 'Operations Analyst']

match = lambda col: eda[col].str.contains('|'.join(keywords), case=False, na=False)

eda['DATA_ANALYST_JOB'] = match('TITLE_NAME') \
             | match('SKILLS_NAME') \
             | match('SPECIALIZED_SKILLS_NAME') 
eda['DATA_ANALYST_JOB'].value_counts()

DATA_ANALYST_JOB
False    37052
True     32148
Name: count, dtype: int64

In [27]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ─────────────────────────────────────────────────────────
# 1) Prepare your data
# ─────────────────────────────────────────────────────────
# assume `eda` is already loaded with DATA_ANALYST_JOB (bool) & NAICS2_NAME
df_grouped = (
    eda
      .groupby(['DATA_ANALYST_JOB','NAICS2_NAME'])
      .size()
      .reset_index(name='Job_Count')
)
# shorten long names...
short_names = {
    'Professional, Scientific, and Technical Services': 'Prof. Services',
    'Administrative and Support and Waste Management and Remediation Services': 'Admin & Waste Mgmt',
    # …etc…
}
df_grouped['Industry'] = df_grouped['NAICS2_NAME'] \
                            .map(short_names) \
                            .fillna(df_grouped['NAICS2_NAME'])
df_grouped['Job_Type'] = df_grouped['DATA_ANALYST_JOB'] \
                             .map({True: 'True', False: 'False'})

# pivot so we have True/False side by side
pivot = df_grouped.pivot_table(
    index='Industry',
    columns='Job_Type',
    values='Job_Count',
    aggfunc='sum',
    fill_value=0
).reset_index()
x       = pivot['Industry']
y_true  = pivot['True']
y_false = pivot['False']

# ─────────────────────────────────────────────────────────
# 2) Build the figure with two rows, but give the top row more height
# ─────────────────────────────────────────────────────────
fig = make_subplots(
    rows=2, cols=1,
    row_heights=[0.80, 0.20],    # 80% height for bars, 20% for table
    specs=[[{'type':'bar'}],[{'type':'table'}]],
    vertical_spacing=0.04       # a little breathing room
)

# add bar traces
colors = {'True':'#FFE5E5','False':'#FF6B6B'}
fig.add_trace(
    go.Bar(
        x=x, y=y_true, name='True',
        marker=dict(color=colors['True'], line=dict(color='#A81D1D',width=1)),
        text=y_true, textposition='outside'
    ),
    row=1, col=1
)
fig.add_trace(
    go.Bar(
        x=x, y=y_false, name='False',
        marker=dict(color=colors['False'],line=dict(color='#A81D1D',width=1)),
        text=y_false, textposition='outside'
    ),
    row=1, col=1
)

# add table down below
fig.add_trace(
    go.Table(
        header=dict(
            values=["Industry","True","False"],
            fill_color='#FDEDEC',
            align='left',
            font=dict(color='#A81D1D',size=13),
            height=30
        ),
        cells=dict(
            values=[x, y_true, y_false],
            fill_color='white',
            align='left',
            font=dict(color='#333',size=11),
            height=25
        )
    ),
    row=2, col=1
)


fig.update_layout(
    title="Data & Business Analytics Job Trends",
    font=dict(family="Helvetica, sans-serif"),

    width=1100, height=750,
    margin=dict(l=60, r=60, t=100, b=100),

    plot_bgcolor='white',
    paper_bgcolor='white',

    xaxis=dict(
        title="Industry",
        title_font=dict(size=16,color='#A81D1D'),
        tickangle=-30,
        linecolor='#A81D1D'
    ),
    yaxis=dict(
        title="Number of Jobs",
        title_font=dict(size=16,color='#A81D1D'),
        gridcolor='rgba(200,200,200,0.3)',
        linecolor='#A81D1D',
        range=[0, max_jobs]
    ),
    legend=dict(
        title="Data Analyst Job",
        x=1.02, y=0.5
    ),

    sliders=[dict(
        active=0,
        currentvalue={"prefix":"Min Jobs: "},
        pad={"t":30},
        x=0.1, y=1.02,          # move the slider to just above the chart
        xanchor='left', yanchor='bottom',
        len=0.6,
        steps=steps
    )]
)

fig.show()
